# Pipeline for model comparsion

please make sure that these two document you have put in the same folder of this ipynd:

For **loading main function** for Softmaxclassifier-forward correction:**softmax_forward_model.py**

For **loading main function** for coteaching with CNN:**coteaching_model.py**

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import torch
# from google.colab import drive
# drive.mount('/content/drive')

# import sys
# sys.path.append("/content/drive/MyDrive/datasets")

import coteaching_model as ct
import softmax_forward_model as sf

from dataclasses import dataclass
@dataclass
class FwdCfg: wd:float=1e-4; max_iter:int=300; seed:int=42

from sklearn.model_selection import train_test_split

# dataset loading
DATASET_1 = "datasets/FashionMNIST0.3.npz"
DATASET_2 = "datasets/FashionMNIST0.6.npz"
DATASET_3 = "datasets/CIFAR.npz"

# DATASET_1 = "/content/drive/MyDrive/datasets/FashionMNIST0.3.npz"
# DATASET_2 = "/content/drive/MyDrive/datasets/FashionMNIST0.6.npz"
# DATASET_3 = "/content/drive/MyDrive/datasets/CIFAR.npz"

DATASET_CONFIGS = [
    {
        "name": "FashionMNIST0.3",
        "path": DATASET_1,
        "coteach": {"lr": 1e-3, "epochs": 10, "wd": 1e-1, "final_forget": 0.3, "sched": "linear", "p_drop": 0.5},
        "forward": {"wd": 0.01, "max_iter": 500},
    },
    {
        "name": "FashionMNIST0.6",
        "path": DATASET_2,
        "coteach": {"lr": 1e-4, "epochs": 30, "wd": 1e-1, "final_forget": 0.75, "sched": "linear", "p_drop": 0.5},
        "forward": {"wd": 0.05, "max_iter": 500},
    },
    {
        "name": "CIFAR",
        "path": DATASET_3,
        "coteach": {"lr": 1e-4, "epochs": 30, "wd": 5e-2, "final_forget": 0.74, "sched": "linear", "p_drop": 0.5},
        "forward": {"wd": 0.05, "max_iter": 500},
    },
]


# Forward confusion matrix


def compute_forward_confusion(dataset_path, model, std):
    d = np.load(dataset_path)
    Xts, Yts = d["Xts"], d["Yts"]
    Xts_f = sf.flatten(Xts)
    Xts_std = std.transform(Xts_f)
    y_pred = model.predict(Xts_std)
    cm = confusion_matrix(Yts, y_pred)
    return cm, Yts, y_pred


def run_pipeline_all(n_runs: int = 10,
                     results_dir: str = "results_runs",
                     val_ratio: float = 0.2):

    os.makedirs(results_dir, exist_ok=True)

    metrics_records = []
    fwd_confusions = {}
    coteach_confusions = {}

    for ds in DATASET_CONFIGS:
        ds_name = ds["name"]
        ds_path = ds["path"]
        print(f"\n========== Dataset: {ds_name} ==========")

        ct_cfg = ds["coteach"]
        fwd_cfg_base = ds["forward"]

        for run in range(n_runs):
            # 1. outdoor layer：seed
            seed = 20 + run
            print(f"\n---- Run {run+1}/{n_runs} (seed={seed}) ----")

            # load data for generating tr_idx / va_idx
            d = np.load(ds_path)
            Xtr, Str, Xts, Yts = d["Xtr"], d["Str"], d["Xts"], d["Yts"]

            # outdoor split for the train/val：
            idx_all = np.arange(len(Str))
            tr_idx, va_idx = train_test_split(
                idx_all,
                test_size=val_ratio,
                stratify=Str,
                random_state=seed,
            )


            # 2 Model Training: Co-Teaching model

            vloss_ct, tacc_ct, m1, m2 = ct.train_config_coteach_scheduled(
                dataset_path=ds_path,
                epochs=ct_cfg["epochs"],
                wd=ct_cfg["wd"],
                lr=ct_cfg["lr"],
                seed=seed,
                final_forget=ct_cfg["final_forget"],
                sched=ct_cfg["sched"],
                p_drop=ct_cfg["p_drop"],
                tr_idx=tr_idx,         # use the outdoor index
                va_idx=va_idx,

            )


            val_acc_ct = 1.0 - vloss_ct
            print(f"[CoTeaching] val_loss={vloss_ct:.4f}, "
                  f"val_acc={val_acc_ct*100:.2f}%, "
                  f"test_acc={tacc_ct*100:.2f}%")

            metrics_records.append({
                "dataset": ds_name,
                "model": "CoTeaching",
                "run": run,
                "seed": seed,
                "val_loss": float(vloss_ct),
                "val_acc": float(val_acc_ct),
                "test_acc": float(tacc_ct),
            })

            # Co-teaching confusion matrix
            DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

            # reshape to [N, C, H, W] for CNN
            if Xts.ndim == 2 and Xts.shape[1] == 28 * 28:
                Xts_img = Xts.reshape(-1, 1, 28, 28)
            elif Xts.ndim == 3:
                Xts_img = Xts[:, None, :, :]
            elif Xts.ndim == 4 and Xts.shape[-1] in (1, 3):
                Xts_img = np.transpose(Xts, (0, 3, 1, 2))
            else:
                raise ValueError(f"Unexpected Xts shape: {Xts.shape}")

            Xts_t = torch.tensor(Xts_img).float().to(DEVICE)

            with torch.no_grad():
                y_pred_1 = m1(Xts_t).argmax(1).cpu().numpy()
                y_pred_2 = m2(Xts_t).argmax(1).cpu().numpy()
            acc1 = (y_pred_1 == Yts).mean()
            acc2 = (y_pred_2 == Yts).mean()
            y_pred_ct = y_pred_1 if acc1 >= acc2 else y_pred_2

            cm_ct = confusion_matrix(Yts, y_pred_ct)
            coteach_confusions[(ds_name, run)] = {
                "cm": cm_ct,
                "y_true": Yts,
                "y_pred": y_pred_ct,
            }


            # 3.Model Training: Forward Model

            cfg = sf.FwdCfg(
                wd=fwd_cfg_base["wd"],
                max_iter=fwd_cfg_base["max_iter"],
                seed=seed,
            )

            # input the tr_idx, va_idx
            val_loss_f, val_acc_f, test_acc_f, model_f, std_f = sf.train_config_forward_once(
                dataset_path=ds_path,
                cfg=cfg,
                tr_idx=tr_idx,
                va_idx=va_idx,
                seed=seed,
            )

            print(f"[Forward ] val_loss={val_loss_f:.4f}, "
                  f"val_acc={val_acc_f*100:.2f}%, "
                  f"test_acc={test_acc_f*100:.2f}%")

            metrics_records.append({
                "dataset": ds_name,
                "model": "Forward",
                "run": run,
                "seed": seed,
                "val_loss": float(val_loss_f),
                "val_acc": float(val_acc_f),
                "test_acc": float(test_acc_f),
            })


            cm_fwd, y_true, y_pred = compute_forward_confusion(ds_path, model_f, std_f)
            fwd_confusions[(ds_name, run)] = {
                "cm": cm_fwd,
                "y_true": y_true,
                "y_pred": y_pred,
            }

        #  summary for each dataset
        print(f"\n====== Summary for dataset: {ds_name} ======")
        df_ds = pd.DataFrame([r for r in metrics_records if r["dataset"] == ds_name])
        for model_name in ["CoTeaching", "Forward"]:
            df_m = df_ds[df_ds["model"] == model_name]
            if len(df_m) == 0:
                continue
            mean_acc = df_m["test_acc"].mean() * 100
            std_acc = df_m["test_acc"].std(ddof=1) * 100
            print(f"{model_name}: mean test acc = {mean_acc:.2f}% "
                  f"(±{std_acc:.2f}%) over {len(df_m)} runs")


    # save the result
    metrics_df = pd.DataFrame(metrics_records)
    metrics_csv_path = os.path.join(results_dir, "metrics.csv")
    metrics_df.to_csv(metrics_csv_path, index=False)
    print(f"\n[Saved] metrics to {metrics_csv_path}")

    fwd_npz_path = os.path.join(results_dir, "forward_confusions.npz")
    np.savez_compressed(
        fwd_npz_path,
        keys=np.array(list(fwd_confusions.keys()), dtype=object),
        cms=np.array([v["cm"] for v in fwd_confusions.values()], dtype=object),
        y_trues=np.array([v["y_true"] for v in fwd_confusions.values()], dtype=object),
        y_preds=np.array([v["y_pred"] for v in fwd_confusions.values()], dtype=object),
    )
    print(f"[Saved] forward confusions to {fwd_npz_path}")

    ct_npz_path = os.path.join(results_dir, "coteach_confusions.npz")
    np.savez_compressed(
        ct_npz_path,
        keys=np.array(list(coteach_confusions.keys()), dtype=object),
        cms=np.array([v["cm"] for v in coteach_confusions.values()], dtype=object),
        y_trues=np.array([v["y_true"] for v in coteach_confusions.values()], dtype=object),
        y_preds=np.array([v["y_pred"] for v in coteach_confusions.values()], dtype=object),
    )
    print(f"[Saved] coteach confusions to {ct_npz_path}")

    return metrics_df, fwd_confusions, coteach_confusions

Device: cpu


In [ ]:
#20 seed的
metrics_df, fwd_confusions, coteach_confusions = run_pipeline_all(
    n_runs=10,
    results_dir="results_runs"
    ,val_ratio=0.2
)


========== Dataset: FashionMNIST0.3 ==========

---- Run 1/10 (seed=20) ----
[CoTeaching] val_loss=0.3125, val_acc=68.75%, test_acc=98.50%
[Forward ] val_loss=0.6654, val_acc=68.53%, test_acc=97.37%

---- Run 2/10 (seed=21) ----
[CoTeaching] val_loss=0.3078, val_acc=69.22%, test_acc=98.33%
[Forward ] val_loss=0.6632, val_acc=68.81%, test_acc=97.40%

---- Run 3/10 (seed=22) ----
[CoTeaching] val_loss=0.3056, val_acc=69.44%, test_acc=98.77%
[Forward ] val_loss=0.6586, val_acc=68.72%, test_acc=97.40%

---- Run 4/10 (seed=23) ----
[CoTeaching] val_loss=0.3075, val_acc=69.25%, test_acc=98.87%
[Forward ] val_loss=0.6639, val_acc=68.92%, test_acc=97.27%

---- Run 5/10 (seed=24) ----
[CoTeaching] val_loss=0.3158, val_acc=68.42%, test_acc=98.13%
[Forward ] val_loss=0.6601, val_acc=68.58%, test_acc=97.40%

---- Run 6/10 (seed=25) ----
[CoTeaching] val_loss=0.3011, val_acc=69.89%, test_acc=98.30%
[Forward ] val_loss=0.6588, val_acc=69.50%, test_acc=97.43%

---- Run 7/10 (seed=26) ----
[CoTeachin